# Caderno para construção e teste do BOT

## Loading

In [76]:
## Loader do PDF
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

path = "data\\base_conhecimento.pdf"

## Loaders structured and unstructured

#loader_un = UnstructuredPDFLoader(path)
loader_structured = PyMuPDFLoader(path)

pages_str = loader_structured.load()
#pages_str = loader_un.load()

## Cleaning

In [77]:
from langchain.text_splitter import CharacterTextSplitter

## Criando texto splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0
    )

## Split
texts = text_splitter.split_documents(pages_str)




In [78]:
### Testanto

texts[500].page_content

'### 5.1 Acessando a Escala\n1. Selecione a escala onde deseja adicionar profissionais adicionais.\n2. Clique com o botão direito e escolha **Adicionar Pessoas Adicionais**.\n### 5.2 Buscando e Adicionando\n1. Busque pelo nome do profissional que deseja adicionar.\n2. Selecione e adicione à escala.\n## Conclusão\nEste guia detalha o processo de adição de profissionais nas escalas do Tasy, incluindo a\nconfiguração de plantões e a geração de escalas. Seguindo os passos descritos, você poderá\notimizar a alocação de profissionais em diferentes horários e tipos de plantão.\n## Metadados\n**Tags**: Sistema Tasy, Escalas, Plantões, Gestão Hospitalar, Adição de Profissionais, Geração de\nEscala, Plantão Noturno, Plantão de Urgência.'

In [79]:
## Função para remover espaço em branco

def remove_ws(d):
    text = d.page_content.replace("\n", " ")
    d.page_content = text
    return d

# Clean texts by removing whitespace from each document
texts_cleaned = [remove_ws(d) for d in texts]

In [80]:
texts_cleaned[500].page_content



'### 5.1 Acessando a Escala 1. Selecione a escala onde deseja adicionar profissionais adicionais. 2. Clique com o botão direito e escolha **Adicionar Pessoas Adicionais**. ### 5.2 Buscando e Adicionando 1. Busque pelo nome do profissional que deseja adicionar. 2. Selecione e adicione à escala. ## Conclusão Este guia detalha o processo de adição de profissionais nas escalas do Tasy, incluindo a configuração de plantões e a geração de escalas. Seguindo os passos descritos, você poderá otimizar a alocação de profissionais em diferentes horários e tipos de plantão. ## Metadados **Tags**: Sistema Tasy, Escalas, Plantões, Gestão Hospitalar, Adição de Profissionais, Geração de Escala, Plantão Noturno, Plantão de Urgência.'

In [81]:
## openai

# config.py
import os
from dotenv import load_dotenv

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path="env/.env")

# Acessar a chave da OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Retriever

In [82]:
## retriever openai

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


# uses OpenAI embeddings to build a retriever
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
# Creates the document retriever using docs and embeddings
db = FAISS.from_documents(texts_cleaned, embeddings)



# Asking the retriever to do similarity search based on Query
#query = "Foreign Aid for Lowari Road Tunnel & Access Roads Project (2nd Revised )"
#answer = db.similarity_search(query)

# Building the retriever
retriever = db.as_retriever(search_kwargs={'k': 3})


In [83]:
import faiss
import pickle

# Salvar o índice FAISS
faiss.write_index(db.index, 'data/faiss_index.index')

# Salvar os documentos associados (metadados)
with open('data/faiss_docs.pkl', 'wb') as f:
    pickle.dump(db.docstore._dict, f)



In [84]:
import faiss
import pickle
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Carregar o índice FAISS
index = faiss.read_index('data/faiss_index.index')

# Carregar os documentos (metadados)
with open('data/faiss_docs.pkl', 'rb') as f:
    docstore = pickle.load(f)

# Recriar o OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# Recriar o FAISS VectorStore usando o índice e os documentos
db = FAISS(embedding_function=embeddings, index=index, docstore=docstore)

# Recriar o retriever
retriever = db.as_retriever(search_kwargs={'k': 3})



TypeError: FAISS.__init__() missing 1 required positional argument: 'index_to_docstore_id'

## Chain and chats

In [97]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Função que o bot usará para responder a mensagens diretas

template = """

Você é Baby, assistente virtual especializada no sistema Tasy de gestão hospitalar. Seu objetivo é fornecer suporte eficiente e preciso aos usuários, ajudando-os a realizar tarefas específicas no sistema.

Leve em consideraçõ sua identidade se for necessário se identificar.

Para dar esse suporte, você tem como material em anexo a transcrição de mais de 300 cursos de como realizar operações no Tasy. 

Nessas transcrições, cada curso é o título no PDF, com os capítulos sendo assinalados com #. Por exemplo:  # Capítulo 2: Cadastro de Motivos de Auditoria,
é o capítulo em que são passados os passos para cadastro de auditoria. SEMPRE SE ATENHA AOS PASSOS DADOS NO MATERIAL.

**Instruções:**
1. Analise cuidadosamente a pergunta do usuário.
2. Identifique o tema principal da pergunta (por exemplo, Cadastro, Faturamento, Prescrição de Medicamentos).
3. Responda diretamente à pergunta com base na base de conhecimento do Tasy.
4. Forneça uma lista clara e numerada de etapas práticas, quando a pergunta for sobre uma operação no sistema.
5. Sugira cursos relevantes, se disponíveis, com base no tema da pergunta.
6. Não passe informações que não estejam na sua base de conhecimento, no {context}. Se não souber a resposta, ou ela for imprecisa, diga:
"Infelizmente não consigo te ajudar com isso! Sugiro que entre em contato com a Comissão do Tasy."
7. Tente orientar a pessoa conversando com você para ter a resposta precisa.
8. Se conseguir identificar qual curso está relacionado àquela tarefa, o indique, por exemplo: "Para mais informações, sugiro acessar o curso 
"FHEMIG - Alta Responsável", no link: https://lector.live/viewer/watch/40de9e01-75af-432c-a781-b9209c2c7951-

**Diretrizes:**
- Use uma linguagem direta e objetiva.
- Priorize precisão e clareza em todas as respostas.
- Se a pergunta for ambígua, peça mais detalhes ao usuário para poder fornecer uma resposta mais precisa.
- Mantenha-se dentro do escopo do sistema Tasy.
- Evite respostas genéricas. Sempre que possível, forneça passos práticos e específicos.
- Se a pergunta estiver ambígua demais, e não conseguir dar uma resposta precisa, peça mais detalhes. Por exemplo, se precisar de ajuda com alguns dos
cursos que são relacionados à temática do faturamento, um tema mais geral, pergunte até ter clareza da resposta a ser dada. Essa clareza deve vir da aderência
com o que está escrito no {context}.

**Exemplo de Resposta para Prescrição de Medicamentos:**
Pergunta: "Como prescrever medicamentos no sistema Tasy?"

Resposta:
Para prescrever medicamentos no sistema Tasy, siga os passos abaixo:
1. Acesse o módulo de Prescrição Eletrônica no menu principal.
2. Selecione o paciente para o qual você deseja prescrever o medicamento.
3. Clique em "Nova Prescrição".
4. Escolha o medicamento a partir da lista disponível ou faça a busca manual pelo nome.
5. Defina a dosagem, frequência e duração do tratamento.
6. Confirme a prescrição e clique em "Salvar".

Dica: Revise a dosagem com cuidado antes de salvar a prescrição.

Para mais informações, recomendo o curso:
- Curso de Prescrição Eletrônica no Tasy [link do curso].

**Fallback para Perguntas Ambíguas:**
Se o modelo não entender completamente a pergunta ou ela for ambígua, peça mais detalhes:
- "Poderia especificar mais detalhes sobre o que você deseja fazer no sistema Tasy? Por exemplo, se está procurando instruções sobre prescrição de medicamentos ou outro processo específico."


A sua base de conhecimento para ajudar o usuário é: {context}

PERGUNTA DO USUÁRIO: {pergunta}

"""

# Converter o template em prompt_template

prompt = ChatPromptTemplate.from_template(template)


## Configurar llm com Ollama

llm = ChatOpenAI(model="gpt-4o")

## Configurar chain

chain = chain = (
# The initial dictionary uses the retriever and user supplied query
    {"context":retriever,
     "pergunta":RunnablePassthrough()}
# Feeds that context and query into the prompt then model & lastly 
# uses the ouput parser, do query for the data.
    |  prompt  | llm | StrOutputParser()
)



## Testando!

In [106]:
input = "como dar alta em um paciente?"


response = chain.invoke(input)
formatted_response = response.replace("\n", "\n\n")  # Adiciona uma linha em branco após cada nova linha

print(formatted_response)


Para dar alta a um paciente no sistema Tasy, siga os passos abaixo:



1. Acesse o módulo de **Prontuário Eletrônico do Paciente**.

2. Navegue até a seção de **Previsão de Alta**.

3. Clique em **Adicionar Previsão de Alta**.

4. Selecione a **probabilidade de alta**:

   - **Alta Confirmada**: Paciente que já realizou o procedimento e deve receber alta em breve.

   - **Alta com Probabilidade**: Escolha entre alta, média ou baixa, conforme a evolução do paciente.

5. Insira a **data prevista de alta**, considerando a evolução clínica atual do paciente.

6. Salve as informações.



Para visualizar ou atualizar previsões de alta, utilize a aba de **Probabilidade de Alta** e aplique os filtros conforme necessário.



Se precisar de mais informações ou ajuda, sugiro acessar cursos específicos relacionados à alta de pacientes no Tasy. 



Caso tenha dúvidas específicas ou necessite de mais detalhes, por favor, entre em contato com a Comissão do Tasy.
